In [20]:
pip install sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 19.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import json

In [3]:
#calling the api for dual eligibility status
url = "https://data.medicaid.gov/api/1/datastore/query/36ed6909-ab49-4ca5-a38e-6790138cd613/0?offset=0&count=true&results=true&schema=true&keys=true&format=json&rowIds=false"


response = requests.get(url)

print(response)

<Response [200]>


In [15]:
#getting it into text so i can load it with json.loads
api= response.text
api

'{"results":[{"state":"Alabama","month":"201601","dualstatus":"Full dual eligibility","countenrolled":"93,611","dqunusable":""},{"state":"Alabama","month":"201601","dualstatus":"Partial dual eligibility","countenrolled":"123,185","dqunusable":""},{"state":"Alabama","month":"201601","dualstatus":"Not dually eligible","countenrolled":"920,630","dqunusable":""},{"state":"Alabama","month":"201602","dualstatus":"Full dual eligibility","countenrolled":"93,259","dqunusable":""},{"state":"Alabama","month":"201602","dualstatus":"Partial dual eligibility","countenrolled":"123,224","dqunusable":""},{"state":"Alabama","month":"201602","dualstatus":"Not dually eligible","countenrolled":"920,173","dqunusable":""},{"state":"Alabama","month":"201603","dualstatus":"Full dual eligibility","countenrolled":"92,894","dqunusable":""},{"state":"Alabama","month":"201603","dualstatus":"Partial dual eligibility","countenrolled":"123,430","dqunusable":""},{"state":"Alabama","month":"201603","dualstatus":"Not dua

In [31]:
#sorting out the data and putting it into a dataframe based on three criteria partial, full, and not dually eligible each being its own dataframe
data = json.loads(api)


filt = [result for result in data['results'] if 'Partial' in result['dualstatus']]
filt2 = [result for result in data['results'] if 'Full' in result['dualstatus']]
filt3 = [result for result in data['results'] if 'No' in result['dualstatus']]

df1 = pd.DataFrame(filt)
df2 = pd.DataFrame(filt2)
df3 = pd.DataFrame(filt3)


df

,state,month,dualstatus,countenrolled,dqunusable
0,Alabama,201601,Full dual eligibility,"93,611",
1,Alabama,201602,Full dual eligibility,"93,259",
2,Alabama,201603,Full dual eligibility,"92,894",
3,Alabama,201604,Full dual eligibility,"92,386",
4,Alabama,201605,Full dual eligibility,"92,018",
...,...,...,...,...,...
3329,Rhode Island,202006,Full dual eligibility,"42,490",
3330,Rhode Island,202007,Full dual eligibility,"42,565",
3331,Rhode Island,202008,Full dual eligibility,"42,736",
3332,Rhode Island,202009,Full dual eligibility,"42,891",


In [51]:
#uploading data to DBeaver
from sqlalchemy import create_engine


user = 'admin'
password = 'isba4715'
host = 'isba-dev-01.cjwugys6yj0u.us-east-1.rds.amazonaws.com'
port_number = '3306'
database_name = 'HPolProject'


connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port_number}/{database_name}'

eng = create_engine(connection_string)

#I am changing the name and the df{} whenever applicable to not make duplicates of the code
tnam = 'MajorEligibilityGroupYr'
df1.to_sql(tnam, eng, if_exists='replace', index=False)


2597

In [41]:
#getting the Major Eligibility Group Information by Month
url2 = "https://data.medicaid.gov/api/1/datastore/query/bffd757a-3ae9-42fc-809a-820c2919496b/0?offset=0&count=true&results=true&schema=true&keys=true&format=json&rowIds=false"


response2 = requests.get(url2)

print(response2)

<Response [200]>


In [46]:
api2

'{"results":[{"state":"Alabama","year":"2016","majoreligibilitygroup":"Children","counteverenrolled":"775,707","countlastmonthenrollment":"764,943","averageenrollmentpermonth":"636,492","dqunusable":""},{"state":"Alabama","year":"2016","majoreligibilitygroup":"Adult expansion group","counteverenrolled":"0","countlastmonthenrollment":"0","averageenrollmentpermonth":"0","dqunusable":""},{"state":"Alabama","year":"2016","majoreligibilitygroup":"Adult","counteverenrolled":"229,568","countlastmonthenrollment":"224,581","averageenrollmentpermonth":"171,306","dqunusable":""},{"state":"Alabama","year":"2016","majoreligibilitygroup":"Aged","counteverenrolled":"125,142","countlastmonthenrollment":"125,142","averageenrollmentpermonth":"112,385","dqunusable":""},{"state":"Alabama","year":"2016","majoreligibilitygroup":"Persons with disabilities","counteverenrolled":"239,639","countlastmonthenrollment":"232,209","averageenrollmentpermonth":"215,413","dqunusable":""},{"state":"Alabama","year":"2016"

In [50]:
#transferring it into a dataframe
api2= response2.text
data2 = json.loads(api2)


filt = [result for result in data2['results'] if '2' in result['year']]



df1 = pd.DataFrame(filt)
df1

,state,year,majoreligibilitygroup,counteverenrolled,countlastmonthenrollment,averageenrollmentpermonth,dqunusable
0,Alabama,2016,Children,"775,707","764,943","636,492",
1,Alabama,2016,Adult expansion group,0,0,0,
2,Alabama,2016,Adult,"229,568","224,581","171,306",
3,Alabama,2016,Aged,"125,142","125,142","112,385",
4,Alabama,2016,Persons with disabilities,"239,639","232,209","215,413",
...,...,...,...,...,...,...,...
2592,Virgin Islands,2022,Adult,"7,439","7,148","6,521",Not assessed
2593,Virgin Islands,2022,Aged,"4,665","4,664","4,260",Not assessed
2594,Virgin Islands,2022,Persons with disabilities,573,519,519,Not assessed
2595,Virgin Islands,2022,COVID newly-eligible,23,20,DS,Not assessed
